In [3]:
use Dan;
use Dan::Pandas;

In [4]:
# Read .csv file (Text::CSV)

use Text::CSV;
my @lines  = csv( in => 'DemographicData.csv' );

my \stats = DataFrame.new( columns => @lines[0], data => @lines[1..*-1] );
~stats[0..3]^;      # head

  Country Name Country Code Birth rate Internet users         Income Group
0        Aruba          ABW     10.244           78.9          High income
1  Afghanistan          AFG     35.253            5.9           Low income
2       Angola          AGO     45.985           19.1  Upper middle income
3      Albania          ALB     12.877           57.2  Upper middle income

In [ ]:
stats.^name;

In [ ]:
# '~' makes a Str, '^' makes a DataFrame from rows

~stats[*-5..*-1]^;    # tail

In [ ]:
stats.shape;

In [ ]:
stats.elems;

In [ ]:
stats.columns.elems;  

In [ ]:
#Clean up #1 - tidy up col labels

stats.pd: ".columns = ['CountryName', 'CountryCode', 'BirthRate', 'InternetUsers', 'IncomeGroup']";
say stats.columns;     #Hash (unordered)
say stats.cx;          #Seq    (ordered)

In [ ]:
## FIXME work this out

##Clean up #2 - coerce cols to numbers      [see DD-raku-Dan-typed.ipynb for why & when]

#stats.data[*;2,3].map({$_.=Rat});
#stats.dtypes;

In [ ]:
# This cell exports the Dan::Pandas::DataFrame for use in DD-raku-Dan-to-seaborn.ipynb

stats.pd: ".to_json('DD-done.json')";

In [ ]:
stats.dtypes;      #FIXME - SO to cell

In [ ]:
stats.describe;    #FIXME - SO to cell

In [ ]:
~stats.series: <BirthRate>;

In [ ]:
~stats[0..5]<CountryName BirthRate>;

In [ ]:
~stats[185..*-1]^;       # Range operator (start .. end)

In [ ]:
#Reverse order
~stats[10,*-1...5]^;     # Sequence operator (start,operation ... end)

In [ ]:
#Every 20th item
~stats[0,20...stats.elems]^;

In [ ]:
#Calculate and Append a new column

my $calcs = Series.new( stats[*]<BirthRate> >>*<< stats[*]<InternetUsers> );   #multiply every row

stats.splice( :ax, *, *, [MyCalc => $calcs] );
~stats[0..5]^;

In [ ]:
#Drop column

stats.splice( :ax, *-1 );
~stats[0..5]^;

In [ ]:
# Grep (check whether column 3 value < 2)

~stats.grep( { .[3] < 2 } );

In [ ]:
# Grep (simple col label check)

~stats.grep( { stats[$++]<InternetUsers> < 2 } );

In [ ]:
# Grep (combine two checks with col labels)

~stats.grep( { my $i=stats[$++]; $i.<InternetUsers> < 2 && $i.<BirthRate> > 40 } );

In [ ]:
# Grep (define filters and apply later)

sub filter1 { $^i.<InternetUsers> < 2  };
sub filter2 { $^i.<BirthRate>     > 40 };

~stats.grep( { my $i=stats[$++]; $i.&filter1 && $i.&filter2 } );

In [ ]:
# Grep with regex

stats.grep( { stats[$++]<IncomeGroup> ~~ /High/ } ).elems;

In [ ]:
# Unique

stats[*]<IncomeGroup>.unique.join(', ');

In [ ]:
# Accessors

stats[4][3];

In [ ]:
stats[4]<InternetUsers>;

In [ ]:
stats<4>[3];   #The default index is letter 4 => 4 (Int) so you may use either [] or <> accessor